In [1]:
import os
import imp
import time
import utils
import torch
import pickle
import datetime
import numpy as np
from tqdm import tqdm
from utils.fcd import *
from utils.data_processor import *
from utils.crop import BrainDataSegCrop
from sklearn.model_selection import KFold

import warnings
warnings.filterwarnings("ignore")

In [2]:
"""
There 3 types of model training:
- whole: on the whole brain 
- temple: on the temple part of the brain
- nottemple: on the whole brain without temple part 
"""
BRAIN_TYPE = 'nottemple' # 'whole', 'temple', 'nottemple'

"""
There 2 types of Local Agragation Operators:
- pospoolxyz
- pointwisemlp
"""
CFG = 'cfgs/brain/brain_pospoolxyz.yaml' # 'cfgs/brain/brain_pointwisemlp.yaml'

"""
There 3 GPUs:
- 0
- 1
- 2
"""
DEVICE = 0
torch.cuda.set_device(DEVICE)
print(f'Current device: {torch.cuda.current_device()}')

FOLDED = True

REPEAT = 1
CROP_SIZE = 64
STEP_SIZE = 32
EPOCHS = 400

IS_RETURN_PC_WITHOUT_AIR_POINTS = False

EXP_NAME = "2021-11-04_16bs_abscoords_air_cropsize64_epochs400_nottemple"
IS_RETURN_ABS_COORDS = "abs" in EXP_NAME


#For loss
LOSS_TYPE = 'BCE'

Current device: 0


In [3]:
config = config_seting(CFG)

with open(f"predictions/KFold/{EXP_NAME}.pkl", 'rb') as f:
    kf = pickle.load(f)

In [4]:
allowed_subjects = np.load('raw_data/sub_with_all_data.npy', allow_pickle=True).tolist()

if not allowed_subjects:
    allowed_subdirs = [f"sub-{name[:-4]}" for name in os.listdir(f"raw_data/normalized_label")]
else:
    allowed_subdirs = [f'sub-{name}' for name in allowed_subjects]
    
brains = [[f"raw_data/output/fmriprep/{subdir}/anat/{name}" for name in os.listdir(f"raw_data/output/fmriprep/{subdir}/anat") if name.endswith("Asym_desc-preproc_T1w.nii.gz")] for subdir in os.listdir(f"raw_data/output/fmriprep/") if subdir.count(".") == 0 and subdir.startswith("sub") and subdir in allowed_subdirs]
brains = [item for sublist in brains for item in sublist]
brain_nums = [x.split('/')[3][4:] for x in brains]

if BRAIN_TYPE == 'whole':
    BRAIN_TYPE = 'full'
    
brains_loaded = [f"dataset_ready_to_use/data_only_usefull_areas/{BRAIN_TYPE}/brains/{num}.npy" for num in tqdm(brain_nums)]
labels_loaded = [f"dataset_ready_to_use/data_only_usefull_areas/{BRAIN_TYPE}/labels/{num}.npy" for num in tqdm(brain_nums)]
curv_loaded = [f"dataset_ready_to_use/data_only_usefull_areas/{BRAIN_TYPE}/curv/{num}.npy" for num in tqdm(brain_nums)]
thickness_loaded = [f"dataset_ready_to_use/data_only_usefull_areas/{BRAIN_TYPE}/thickness/{num}.npy" for num in tqdm(brain_nums)]
sulc_loaded = [f"dataset_ready_to_use/data_only_usefull_areas/{BRAIN_TYPE}/sulc/{num}.npy" for num in tqdm(brain_nums)]

100%|██████████| 81/81 [00:00<00:00, 402057.54it/s]


In [5]:
!mkdir predictions/{EXP_NAME}

In [6]:
def model_and_brains_to_points_with_predictions(config,
                                                kf,
                                                num_votes,
                                                repeat,
                                                brains_loaded,
                                                labels_loaded,
                                                curv_loaded,
                                                thickness_loaded,
                                                sulc_loaded,
                                                exp_name
                                               ):
    """
    """
    
    res = {}
    for e, (train_idxs, test_idxs) in tqdm(enumerate(kf.split(brains_loaded))): 
        
        test_dicts = [{'brains': [brains_loaded[idx]],
                       'labels': [labels_loaded[idx]],
#                        'curvs': [curv_loaded[idx]],
#                        'thickness': [thickness_loaded[idx]],
#                        'sulc': [sulc_loaded[idx]]
                      } for idx in test_idxs]
        
        model, criterion = build_multi_part_segmentation(config=config, weights=None, type=LOSS_TYPE)
        model.load_state_dict(torch.load(f'checkpoints/{exp_name}_{e + 1}_fold.pth'))
        model.cuda()
        model.eval()
        criterion.cuda()
        
        with torch.no_grad():
            
            transform_for_vote = d_utils.BatchPointcloudScaleAndJitter(scale_low=config.scale_low,
                                                                       scale_high=config.scale_high,
                                                                       std=config.noise_std,
                                                                       clip=config.noise_clip
                                                                      )
            
            softmax = torch.nn.Softmax(dim=0)
            
            data_loader = [BrainDataSegCrop(num_points=4096,
                                            task='test',
                                            data_dict = test_dict,
                                            step_size = STEP_SIZE,
                                            crop_size = CROP_SIZE,
                                            return_center = True,
                                            return_air_mask_test = True,
                                            is_folded = FOLDED,
                                            return_pc_without_air_points = IS_RETURN_PC_WITHOUT_AIR_POINTS, 
                                            return_abs_coords = IS_RETURN_ABS_COORDS,
                                            MEANS = {'brains': 161.2455},
                                            STDS = {'brains': 167.36417}
                                           ) for test_dict in test_dicts]
            
            for i, test_brain in tqdm(enumerate(test_idxs)):
                print(f"Working on brain {test_brain}")
                pred_soft_flats = []
                air_masks_flats = []
                points_labels_flats = []
                center_coords_flats = []
                points_orig_flats = []
                points_masks_flats = []#NNN
                all_logits = []
                all_points_labels = []
                all_shape_labels = []
                all_masks = []
                print(len(data_loader[i]))
                for k, crop in enumerate(data_loader[i]):
                    for _ in range(repeat):
                        points_orig, mask, points_labels, shape_labels, air_mask = [
                            crop[key] for key in [
                                "current_points", "mask", "current_points_labels", "label",'air_mask'
                            ]
                        ]
                        vote_logits = None
                        vote_points_labels = None
                        vote_shape_labels = None
                        vote_masks = None
                        points_orig = points_orig.unsqueeze(0)
                        mask = mask.unsqueeze(0)
                        points_labels = points_labels.unsqueeze(0)
                        shape_labels = shape_labels.unsqueeze(0)
                        preds = []
                        for v in range(num_votes):
                            batch_logits = []
                            batch_points_labels = []
                            batch_shape_labels = []
                            batch_masks = []
                            if v > 0:
                                points = transform_for_vote(points_orig)
                            else:
                                points = points_orig
                            # forward
                            features = points
                            features = features.transpose(1, 2).contiguous()
                            points = points[:, :, :3].cuda(non_blocking=True)
                            mask = mask.cuda(non_blocking=True)
                            features = features.cuda(non_blocking=True)
                            points_labels = points_labels.cuda(non_blocking=True)
                            shape_labels = shape_labels.cuda(non_blocking=True)
                            pred = model(points, mask, features)
                            preds.append(pred[0])
                        preds = torch.cat(preds).mean(dim=0)
                        
                        points_orig = points_orig.squeeze(0)                        
                        pred_soft_flats += softmax(preds)[1,:].reshape(-1).detach().cpu().numpy().tolist()
                        points_labels_flats += points_labels.reshape(-1).detach().cpu().numpy().tolist()
                        air_masks_flats += air_mask.reshape(-1).tolist()
                        if IS_RETURN_ABS_COORDS:
                            size_abs = (241, 336, 283)
                            means = np.array([x // 2 for x in size_abs])
                            half_range = np.array([x // 2 for x in size_abs])
                            points_orig_flats += (points_orig[:, :3].detach().cpu().numpy() * half_range + means).tolist()
                            
                        else:
                            points_orig_flats += (((points_orig[:, :3]) * (CROP_SIZE//2) + (CROP_SIZE//2)).detach().cpu().numpy() + np.array(center_coords)).tolist()

                np.save(f"predictions/{exp_name}/brain{test_brain}", (points_orig_flats, pred_soft_flats, points_labels_flats,air_masks_flats), allow_pickle=True)
                print(k)
                print(f"brain {test_brain} done")
                res[test_brain] = k
            del data_loader


In [7]:
model_and_brains_to_points_with_predictions(config=config,
                                                kf=kf,
                                                num_votes=1,
                                                repeat=1,
                                                brains_loaded=brains_loaded,
                                                labels_loaded=labels_loaded,
                                                curv_loaded=curv_loaded,
                                                thickness_loaded=thickness_loaded,
                                                sulc_loaded = sulc_loaded,
                                                exp_name = EXP_NAME)

0it [00:00, ?it/s]
0it [00:00, ?it/s]

Working on brain 0
49



1it [00:11, 11.29s/it]

48
brain 0 done
Working on brain 4
49



2it [00:22, 11.24s/it]

...

48
brain 71 done
Working on brain 74
48



15it [02:47, 11.17s/it]

47
brain 74 done
Working on brain 75
49



16it [02:59, 11.19s/it]
5it [18:11, 218.35s/it]

48
brain 75 done


In [10]:
!nvidia-smi

Fri Oct 29 08:18:55 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64       Driver Version: 440.64       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:01:00.0 Off |                  N/A |
| 20%   33C    P8    16W / 250W |     10MiB / 11177MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:02:00.0 Off |                  N/A |
| 20%   31C    P2    54W / 250W |  11174MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [10]:
import numpy as np
from tqdm import tqdm

In [15]:
if BRAIN_TYPE == 'full':
    res_dict = {}
    for file in glob.glob(f'predictions/{EXP_NAME}/*'):
        a = np.load(file, allow_pickle=True)
        brain = int(file.split('/')[-1].split('brain')[-1].split('.npy')[0])
        res_dict[brain] = a
elif BRAIN_TYPE == 'temple_nottemple':
    res_dict = {}
    for num in range(len(brain_nums)):
        file1 = f'predictions/{EXP_NAME1}/brain{num}.npy'
        file2 = f'predictions/{EXP_NAME2}/brain{num}.npy'
        a = np.load(file1, allow_pickle=True)
        b = np.load(file2, allow_pickle=True)
        c = []
        for i in range(len(a)):
            c.append(np.concatenate([a[i],b[i]]))
        res_dict[num] = c
        np.save(f'predictions/{EXP_NAME}/brain{num}.npy',c)

In [16]:
def result_to_metrics(res, th = 0.5):
    pred = res[1]
    true = res[2]
    mask = res[3]
    pred = [pred[i] for i in range(len(mask)) if mask[i]==0]
    true = [true[i] for i in range(len(mask)) if mask[i]==0]
    if np.sum(true) == 0:
        return -999, -999, -999, -999, -999, -999
    pred = np.array(pred)
    conf = confusion_matrix(true, pred > th)
    IoU = conf[1,1] / (conf[1, 1] + conf[1, 0] + conf[0,1])
    dice = 2 * conf[1,1] / (2 * conf[1, 1] + conf[1, 0] + conf[0, 1])
    d_in = np.dot(true, pred) / np.sum(true)
    d_out = np.dot(np.ones(len(true)) - np.array(true), pred) / (len(true) - np.sum(true))
    contrast = (d_in - d_out) /  (d_in + d_out)
    roc = roc_auc_score(true, pred)
    recall = conf[1,1] / (conf[1, 1] + conf[1, 0])
    return conf, IoU, dice, contrast, roc, recall


def res_dict_to_metrics(res_dict, th = 0.5):
    res_metrics = {}
    for brain in tqdm(res_dict):
        res_metrics[brain] = {}
        res = res_dict[brain]
        res_metrics[brain]['conf_all'],res_metrics[brain]['IoU_all'],res_metrics[brain]['dice_all'],res_metrics[brain]['contrast_all'],res_metrics[brain]['roc_all'],res_metrics[brain]['recall_all'] = result_to_metrics(res,th)
        len_1_part = len(res[0]) // REPEAT
        res_metrics[brain]['confs'] = []
        res_metrics[brain]['IoUs'] = []
        res_metrics[brain]['dices'] = []
        res_metrics[brain]['contrasts'] = []
        res_metrics[brain]['roc'] = []
        res_metrics[brain]['recall'] = []
        for repeat in range(REPEAT):
            res_part = [res[i][len_1_part * repeat:len_1_part * (repeat + 1)] for i in range(4)]
            conf, IoU, dice, contrast, roc, recall = result_to_metrics(res_part, th)
            res_metrics[brain]['confs'].append(conf)
            res_metrics[brain]['IoUs'].append(IoU)
            res_metrics[brain]['dices'].append(dice)
            res_metrics[brain]['contrasts'].append(contrast)
            res_metrics[brain]['roc'].append(roc)
            res_metrics[brain]['recall'].append(recall)
    return res_metrics

In [17]:
final = {}
for th in [0.5]:#np.linspace(0.001,0.999,35):
    res_metrics=res_dict_to_metrics(res_dict,th)
    final[th] = res_metrics


100%|██████████| 17/17 [00:06<00:00,  2.58it/s]


In [18]:
for th in [0.5]:#np.linspace(0.001,0.999,35):
    tmp = []
    for brain in final[th]:
        tmp.append(final[th][brain]['dice_all'])
    print(th, np.mean(tmp))

0.5 -176.2890909980588


In [28]:
print(EXP_NAME)
for th in [0.5]:
    for brain in final[th]:
        print(str(final[th][brain]['contrast_all']).replace('.',','))

2021-10-16_16bs_abscoords_air_cropsize64_epochs400_whole
-0,5453059781310424
-999
0,16212565185072944
0,3254029387266164
0,289756800487056
-0,07076440306089676
-0,20348429852306948
-999
0,35766872661995175
0,28154538878747326
-999
0,08520021962822351
-0,23943755223604346
0,20373747463776357
0,08962772003588299
0,27886874113518345
0,025790812399981825


In [23]:
final[th]

{22: {'conf_all': array([[32156, 21070],
         [    6,     0]]),
  'IoU_all': 0.0,
  'dice_all': 0.0,
  'contrast_all': -0.5453059781310424,
  'roc_all': 0.11893310286952491,
  'recall_all': 0.0,
  'confs': [array([[32156, 21070],
          [    6,     0]])],
  'IoUs': [0.0],
  'dices': [0.0],
  'contrasts': [-0.5453059781310424],
  'roc': [0.11893310286952491],
  'recall': [0.0]},
 30: {'conf_all': -999,
  'IoU_all': -999,
  'dice_all': -999,
  'contrast_all': -999,
  'roc_all': -999,
  'recall_all': -999,
  'confs': [-999],
  'IoUs': [-999],
  'dices': [-999],
  'contrasts': [-999],
  'roc': [-999],
  'recall': [-999]},
 49: {'conf_all': array([[38101, 13042],
         [   55,    44]]),
  'IoU_all': 0.0033482992161935924,
  'dice_all': 0.006674251042851726,
  'contrast_all': 0.16212565185072944,
  'roc_all': 0.661540714617382,
  'recall_all': 0.4444444444444444,
  'confs': [array([[38101, 13042],
          [   55,    44]])],
  'IoUs': [0.0033482992161935924],
  'dices': [0.0066742